In [1]:
# %%
import torch
from torch.utils.data import Dataset, DataLoader
import xarray as xr
import numpy as np
from scipy.spatial import cKDTree
from scipy.interpolate import griddata
import pandas as pd
import time
import os

In [2]:
# Load NYSM station data
nysm = pd.read_csv('nysm.csv')
# NYSM station lat/lon
nysm_latlon = np.stack([
    nysm['lat [degrees]'].values,
    (nysm['lon [degrees]'].values + 360) % 360
], axis=-1)

In [20]:
# %%
# Loading the NYSM station data
NYSM = xr.open_dataset('data/NYSM.nc')
NYSM['longitude'] = (NYSM['longitude']+360) % 360   # this is needed to match the RTMA lon
NYSM_lat = NYSM.latitude.values
NYSM_lon = NYSM.longitude.values
# Precompute grid KDTree
station_points = np.stack([NYSM_lat.ravel(), NYSM_lon.ravel()], axis=-1)
tree = cKDTree(station_points)
# Query the station locations
_, station_indices = tree.query(nysm_latlon)
NYSM = NYSM.isel(station=station_indices)  # this is needed to match the nysm_latlon order

In [21]:
# Checking the missing values across all the years
print(f'Number of total instances in NYSM: {len(NYSM.time)}')
for var in ['i10fg','si10','t2m','sh2']:
    for i in range(1,len(NYSM['station'])+1):
        missing_condition = NYSM[var].isnull().sum(dim='station') > i 
        missing_instances = len(NYSM['time'].where(missing_condition).dropna(dim='time'))
        if missing_instances == 0:
            print(f'for {var}, {len(NYSM['station'])-i} number of stations are showing no missing instances.')
            break

Number of total instances in NYSM: 631008
for i10fg, 99 number of stations are showing no missing instances.
for si10, 99 number of stations are showing no missing instances.
for t2m, 95 number of stations are showing no missing instances.
for sh2, 17 number of stations are showing no missing instances.


# Identifying missing instances across any station

In [277]:
# Checking the missing values during 2023, at every hour
NYSM_subset = NYSM.sel(time=slice('2023-01-01T00:00', '2023-12-31T00:00'))
NYSM_subset = NYSM_subset.resample(time='1h').nearest()
print(f'Number of total instances in NYSM: {len(NYSM_subset.time)}')
all_missing_instances = []
for var in ['i10fg','si10','t2m','sh2']:
    missing_condition = NYSM_subset[var].isnull().any(dim='station')
    missing_instances = NYSM_subset['time'].where(missing_condition).dropna(dim='time')
    print(f'for {var}, {len(missing_instances)} number of missing instances exist.')
    all_missing_instances.append(missing_instances.time)
all_missing_instances = xr.concat(all_missing_instances, dim='time').drop_duplicates('time')
print( f' Total number of missing instances: {len(all_missing_instances)}, and non-missing instances: {len(NYSM_subset.time) - len(all_missing_instances)}, during 2023.')

Number of total instances in NYSM: 8737
for i10fg, 1679 number of missing instances exist.
for si10, 1679 number of missing instances exist.
for t2m, 3841 number of missing instances exist.
for sh2, 5815 number of missing instances exist.
 Total number of missing instances: 5853, and non-missing instances: 2884, during 2023.


# Identifying number of stations with no missing instances

In [262]:
# Checking the missing values during 2023, at every hour
NYSM_subset = NYSM.sel(time=slice('2023-01-01T00:00', '2023-12-31T00:00'))
NYSM_subset = NYSM_subset.resample(time='1h').nearest()
print(f'Number of total instances in NYSM: {len(NYSM_subset.time)}')
for var in ['i10fg','si10','t2m','sh2']:
    missing_condition = NYSM_subset[var].isnull().sum(dim='station')    # for each time step, how many stations are missing
    max_missing_stations = missing_condition.sortby(missing_condition, ascending=False)[0]  # sorting and getting the maximum number of stations missing during a same time instance
    print(f'For {var}, Maximum number of stations missing during a same time instance: {max_missing_stations.values}')
    # This number of stations we can deselect randomly during each instance. 

Number of total instances in NYSM: 8737
For i10fg, Maximum number of stations missing during a same time instance: 9
For si10, Maximum number of stations missing during a same time instance: 9
For t2m, Maximum number of stations missing during a same time instance: 31
For sh2, Maximum number of stations missing during a same time instance: 31


- Though the above script says morethan 95 stations are having non missing instances, the stations are not consistant across the times. 
- There is not a single station that doesn't have none values. 
- It is very hard to identify commnly missing stations, during all instances. 
- Rather, we can select the required number of stations that are having maximum number of missing time instances. 

In [ ]:
# Checking the missing values during 2023, at every hour
NYSM_subset = NYSM.sel(time=slice('2023-01-01T00:00', '2023-12-31T00:00'))
NYSM_subset = NYSM_subset.resample(time='1h').nearest()
print(f'Number of total instances in NYSM: {len(NYSM_subset.time)}')

all_missing_stations = []
for var in ['i10fg','si10','t2m','sh2']:
    missing_condition = NYSM_subset[var].isnull().sum(dim='station')    # for each time step, how many stations are missing
    max_missing_stations = missing_condition.sortby(missing_condition, ascending=False)[0]  # sorting and getting the maximum number of stations missing during a same time instance
    
    missing_condition = NYSM_subset[var].isnull().sum(dim='time')    # for each time step, how many stations are missing
    stations_with_max_missing_instances = missing_condition.sortby(missing_condition, ascending=False)[0:max_missing_stations.values]  # sorting and getting the stations with maximum missing instances
    print(f'For {var}, Stations with maximum missing instances are: {stations_with_max_missing_instances.station.values}')
    # These are the stations we can deselect randomly during each instance. 
    all_missing_stations.append(stations_with_max_missing_instances.station)

all_missing_stations = xr.concat(all_missing_stations, dim='station').drop_duplicates('station')

print( f' Total number of stations with maximum missing instances: {len(all_missing_stations)}, and non-missing stations: {len(NYSM_subset.station) - len(all_missing_stations)}, during 2023.')

Number of total instances in NYSM: 8737
For i10fg, Stations with maximum missing instances are: ['CROG' 'SCHU' 'DUAN' 'DEPO' 'CLAR' 'ELDR' 'RAND' 'WEST' 'TUPP']
For si10, Stations with maximum missing instances are: ['CROG' 'SCHU' 'DUAN' 'DEPO' 'CLAR' 'ELDR' 'RAND' 'WEST' 'TUPP']
For t2m, Stations with maximum missing instances are: ['CROG' 'CHAZ' 'CLYM' 'DUAN' 'POTS' 'CINC' 'CLAR' 'BSPA' 'BATA' 'SCHU'
 'WALT' 'WALL' 'FAYE' 'SHER' 'TUPP' 'CSQR' 'WOLC' 'ELDR' 'WEST' 'SBRI'
 'CAPE' 'SCHO' 'MALO' 'EDWA' 'OLDF' 'KIND' 'DEPO' 'HARP' 'GFAL' 'COHO'
 'RAND']
For sh2, Stations with maximum missing instances are: ['CROG' 'TICO' 'BING' 'CLYM' 'CHAZ' 'CINC' 'LAUR' 'OWEG' 'DUAN' 'CLAR'
 'POTS' 'SOUT' 'BSPA' 'KIND' 'BATA' 'SCHU' 'WALT' 'SHER' 'WALL' 'FAYE'
 'TUPP' 'CSQR' 'WOLC' 'EAUR' 'ELDR' 'WEST' 'SBRI' 'BELD' 'CAPE' 'TANN'
 'OLDF']
 Total number of stations with maximum missing instances: 39, and non-missing stations: 87, during 2023.


- Since, there is no station that is not missing during any time instance, it is better to train a model with random stations at every instance. 